# Comparison of spectrum between GIGATL outputs and Alitka Satellite 

Requisites :
 - git clone https://github.com/auraoupa/gonzag_cloud
 


In [1]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
GONZAG_DIR = '/home/jovyan/gonzag_cloud/gonzag' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(GONZAG_DIR)
import gonzag as gz


## Params

For the model

In [3]:
model = 'GIGATL'
region = '1'
season = 'fma'
name_mod = model+'-Region'+region+'-'+season
name_ssh_mod = 'zeta'
name_lsm_mod = 'salt' 
name_lat_mod = 'lat'
name_lon_mod = 'lon'
name_time_mod = 'ocean_time'
l_griddist = False
period_mod = ['2000-02-01','2000-04-30']

For altimetry data

In [4]:
name_sat= 'Altika'
tag_sat_catalog = 'al'
name_ssh_sat='sla_unfiltered'
name_time_sat='time'
period_sat = ['2014-02-01','2014-04-30'] #should cover the same period of a different year if needed

## Data

In [5]:
import xarray as xr
import intake
import gcsfs

gcs = gcsfs.GCSFileSystem(requester_pays=True)

cat = intake.open_catalog('../SWOT-AdAC/catalog.yaml')
dsmod1 = cat[model](region=region,depth='surf', season=season).to_dask()

In [6]:
dsmod1

<xarray.Dataset>
Dimensions:     (eta_rho: 1489, eta_v: 1488, time: 2280, xi_rho: 1491, xi_u: 1490)
Dimensions without coordinates: eta_rho, eta_v, time, xi_rho, xi_u
Data variables: (12/13)
    angle       (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    f           (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    h           (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    lat         (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    lon         (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    ocean_time  (time) float32 dask.array<chunksize=(24,), meta=np.ndarray>
    ...          ...
    pn          (eta_rho, xi_rho) float32 dask.array<chunksize=(1489, 1491), meta=np.ndarray>
    salt        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>
    temp        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>
    u           (time, eta_rho, xi_u) float32 dask.array<chunksize=(24, 1489, 1490), meta=np.ndarray>
    v           (time, eta_v, xi_rho) float32 dask.array<chunksize=(24, 1488, 1491), meta=np.ndarray>
    zeta        (time, eta_rho, xi_rho) float32 dask.array<chunksize=(24, 1489, 1491), meta=np.ndarray>

In [7]:
#get rid of 29th of february (2010 = leap year)
import numpy as nmp
from gonzag.io import GetTimeVector
from gonzag.utils import scan_idx
rvt = GetTimeVector( dsmod1, name_time_mod, lquiet=True )
(rtu1,rtu2)=nmp.array(['2000-02-29T00:00:00','2000-03-01T00:00:00'], dtype='datetime64')
jt1, jt2 = scan_idx( rvt, rtu1, rtu2 )
dsmod=dsmod1.drop_isel(time=nmp.arange(jt1,jt2+1,1))

In [8]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/altimetry.yaml")
dssat  = cat[tag_sat_catalog].to_dask()

In [9]:
dssat

<xarray.Dataset>
Dimensions:         (time: 34340107)
Coordinates:
    latitude        (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
  * time            (time) datetime64[ns] 2013-03-14T05:44:50.445185 ... 2015...
Data variables:
    cycle           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
    dac             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    lwe             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    mdt             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    ocean_tide      (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_filtered    (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    track           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
Attributes: (12/24)
    Conventions:               CF-1.6
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    cdm_data_type:             Swath
    comment:                   Sea surface height measured by altimeters refe...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    creator_email:             servicedesk.cmems@mercator-ocean.eu
    ...                        ...
    software_version:          6.2_DUACS_DT2018_baseline
    source:                    Altika measurements
    ssalto_duacs_comment:      The reference mission used for the altimeter i...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    summary:                   SSALTO/DUACS Delayed-Time Level-3 sea surface ...
    title:                     DT Altika Global Ocean Along track SSALTO/DUAC...

## Model and Satellite objects

In [10]:
ModelGrid = gz.ModGrid( dsmod, period_mod, name_lon_mod, name_lat_mod , name_time_mod, dsmod, name_lsm_mod, distorded_grid=False )



 *** what we use to define model land-sea mask:
    => "salt" in dataset 

 *** Skipping computation of angle distortion of the model grid! ("-D" option not invoked)...

 *** About model gridded (source) domain:
     * shape =  (1489, 1491)
     * horizontal resolution:  0.007546403  degrees or  0.8384808350680396  km
     * Is this a global domain w.r.t longitude:  False
       ==> this is a regional domain
       ==> working in the [0:360] frame...
     * lon_min, lon_max =  279.15 298.04
     * lat_min, lat_max =  25.86 42.41
     * should we pay attention to possible STRONG local distorsion in the grid:  False
     * number of time records of interest for the interpolation to come:  2112
       ==> time record dates: 2000-02-01 to 2000-04-30, included



In [11]:
SatelliteTrack = gz.SatTrack( dssat, period_sat, name_time_sat, name_ssh_sat, domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )

 *** [SatTrack()] Analyzing the time vector in dataset ...

 *** About satellite track (target) domain:
     * number of time records of interest for the interpolation to come:  29724
       ==> time record indices: 14883510 to 19033581, included

       separated in 154 tracks


## Process the tracks in parallel (results are saved in separate netcdf files)

In [12]:
if not os.path.exists('results_'+name_sat+'-'+name_mod):
        os.makedirs('results_'+name_sat+'-'+name_mod)

In [13]:
def process_one_track(track):
    tt = "{:02d}".format(track)
    Solution0 = gz.Model2SatTrack( ModelGrid, name_ssh_mod, SatelliteTrack, name_ssh_sat, track )
#    c1     = 'Model SSH interpolated in space (' ; c2=') and time on satellite track'
#    vvar   = [ 'latitude', 'longitude', name_ssh_mod+'_np'   , name_ssh_mod+'_bl' , name_ssh_sat          , 'distance'                            ]
#    vunits = [ 'deg.N'   , 'deg.E'    ,          'm'         ,     'm'            ,    'm'                ,    'km'                               ]
#    vlongN = [ 'Latitude', 'Longitude', c1+'nearest-point'+c2,  c1+'bilinear'+c2  , 'Input satellite data', 'Cumulated distance from first point' ]
#    from gonzag.config import rmissval
#    iw = gz.io.SaveTimeSeries( Solution0.time, \
#                             nmp.array( [Solution0.lat, Solution0.lon, Solution0.ssh_mod_np,
#                                         Solution0.ssh_mod, Solution0.ssh_sat, Solution0.distance] ), \
#                             vvar, 'results_'+name_sat+'-'+name_mod+'/result_'+str(tt)+'.nc', time_units='',\
#                             vunits=vunits, vlnm=vlongN, missing_val=rmissval )
    return Solution0


In [14]:
track=0
sol0=process_one_track(track)

In [15]:
sol0.lat

<xarray.Variable (time: 102)>
dask.array<getitem, shape=(102,), dtype=float64, chunksize=(102,), chunktype=numpy.ndarray>
Attributes:
    long_name:      Latitude of measurement
    standard_name:  latitude
    units:          degrees_north

In [16]:
sol0.ssh_mod_np

masked_array(data=[--, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --, --, --, --, --, --, --, --, --, --, --,
                   --, --, --, --],
             mask=[ True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True,  Tr

In [17]:
(MG, name_ssh_mod, ST, name_ssh_sat, one_track ) = (ModelGrid, name_ssh_mod, SatelliteTrack, name_ssh_sat, track)

In [19]:
import time ; # to report execution speed of certain parts of the code...
import pandas as pd
import numpy as np
from gonzag.config import ldebug, ivrb, nb_talk, l_plot_meshes, deg2km, rfactor, search_box_w_km, l_save_track_on_model_grid, l_plot_meshes, rmissval
from gonzag.utils  import *
from gonzag.bilin_mapping import BilinTrack


In [21]:
        from gonzag.io   import GetModel2DVar, GetSatSSH

        jt1=ST.index_tracks[one_track][0]
        jt2=ST.index_tracks[one_track][1]

        (Nj,Ni) = MG.shape

        d_found_km = rfactor*MG.HResDeg*deg2km
        print(' *** "found" distance criterion when searching for nearest point on model grid is ', d_found_km, ' km\n')

        # Size of the search zoom box:
        np_box_radius = SearchBoxSize( MG.HResDeg*deg2km, search_box_w_km )
        print(' *** Will use zoom boxes of width of '+str(2*np_box_radius+1)+' points for 1st attempts of nearest-point location...\n')

        Nt = jt2 - jt1 ; # number of satellit observation point to work with here...



 *** "found" distance criterion when searching for nearest point on model grid is  0.6288606263010297  km

 *** Will use zoom boxes of width of 597 points for 1st attempts of nearest-point location...



In [23]:
        if_talk = Nt//nb_talk

        BT = BilinTrack( ST.lat[jt1:jt2], ST.lon[jt1:jt2], MG.lat, MG.lon, src_grid_local_angle=MG.xangle, k_ew_per=MG.EWPer, rd_found_km=d_found_km, np_box_r=np_box_radius, freq_talk=if_talk )


In [25]:
print(jt1,jt2)

203 305


In [26]:
for jt in range(jt1,jt2):
    PlotMesh( (ST.lon[jt],ST.lat[jt]), MG.lat, MG.lon, BT.SM[jt-jt1,:,:], BT.WB[jt-jt1,:], \
                                  fig_name='mesh_jt'+'%5.5i'%(jt)+'.png' )

In [27]:
        vssh_m_np = nmp.zeros(Nt) ; vssh_m_np[:] = rmissval; # vector to store the model data interpolated in time and space (nearest-point) on the satellite track...
        vssh_m_bl = nmp.zeros(Nt) ; vssh_m_bl[:] = rmissval; # vector to store the model data interpolated in time and space (bilinear) on the satellite track...
        vdistance = nmp.zeros(Nt)

        # Time increment on the satellite time:
        ktm1   = 0   ; ktm2   = 0
        ktm1_o = -10 ; ktm2_o = -10


In [28]:
jt=jt1

In [29]:
            itt = ST.time[jt] ; # unix time
            year_sat=pd.to_datetime(itt).year
            date_model=pd.Series(MG.time)
            date_model_satyear=date_model.apply(lambda dt: dt.replace(year=year_sat))

            # Get surrounding records for model:
            kt = ktm1
            while not (date_model_satyear[kt]<=itt and date_model_satyear[kt+1]>itt): kt=kt+1
            ktm1 = kt ; ktm2 = kt+1


In [30]:
kt

34

In [32]:
date_model_satyear[kt]

Timestamp('2014-02-02 10:29:20')

In [33]:
itt

numpy.datetime64('2014-02-02T10:58:18.622069000')

In [34]:
            if (ktm1>ktm1_o) and (ktm2>ktm2_o):
                if (ktm1_o == -10) or (ktm1 > ktm2_o):
                    if ivrb>0: print(' *** Reading '+name_ssh_mod+' in model dataset \n    => at ktm1=', ktm1)
                    Xm1 = GetModel2DVar( MG.file, name_ssh_mod, kt=ktm1 )
                else:
                    Xm1[:,:] = Xm2[:,:]
                #
                if ivrb>0: print(' *** Reading '+name_ssh_mod+' in in model dataset \n    => at ktm2=', ktm2)
                Xm2 = GetModel2DVar( MG.file, name_ssh_mod, kt=ktm2 )

                # slope only needs to be calculated when Xm2 and Xm1 have been updated:
                Xa = (Xm2 - Xm1) / float((MG.time[ktm2] - MG.time[ktm1] )/ np.timedelta64(1, 's'))

In [35]:
Xm1

<xarray.DataArray 'zeta' (eta_rho: 1489, xi_rho: 1491)>
array([[0.6456985 , 0.6439263 , 0.6420803 , ..., 0.6580962 , 0.6582699 ,
        0.658002  ],
       [0.6459294 , 0.64411724, 0.6422069 , ..., 0.65521467, 0.6550452 ,
        0.6545457 ],
       [0.6460821 , 0.64426327, 0.64234227, ..., 0.6519498 , 0.65152955,
        0.6509034 ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)
Dimensions without coordinates: eta_rho, xi_rho
Attributes:
    long_name:  averaged SSH
    units:      m

In [36]:
Xm2

<xarray.DataArray 'zeta' (eta_rho: 1489, xi_rho: 1491)>
array([[0.7517791 , 0.7508314 , 0.7498385 , ..., 0.7337062 , 0.7339012 ,
        0.73366994],
       [0.7520626 , 0.7510511 , 0.74999034, ..., 0.73080444, 0.73076093,
        0.73028195],
       [0.7524308 , 0.7513339 , 0.7501734 , ..., 0.7278422 , 0.72761655,
        0.7269531 ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)
Dimensions without coordinates: eta_rho, xi_rho
Attributes:
    long_name:  averaged SSH
    units:      m

In [38]:
            Xm = Xm1[:,:] + Xa[:,:]*float((itt - date_model_satyear[ktm1])/ np.timedelta64(1, 's'))

            [ [j1,i1],[j2,i2],[j3,i3],[j4,i4] ] = BT.SM[jt-jt1,:,:]
            [w1, w2, w3, w4]                    = BT.WB[jt-jt1,:]

            Sm = MG.mask[j1,i1] + MG.mask[j2,i2] + MG.mask[j3,i3] + MG.mask[j4,i4]

In [40]:
Sm.values

array(144)

In [41]:
MG.mask[j1,i1].values

array(36)

In [47]:
import matplotlib.pyplot as plt 
MG.mask.plot()
plt.savefig('plot_mask.png')

In [48]:
(gridset, varlsm)=(dsmod, name_lsm_mod)

( dataset, what )=( gridset, varlsm )

In [49]:
    print('\n *** what we use to define model land-sea mask:\n    => "'+what+'" in dataset \n')
    l_fill_val = (what[:10]=='_FillValue')
    l_nonzero_val = (what[:10]=='_IsNotZero')
    ncvar = what
    if l_fill_val: ncvar = what[11:]
    if l_nonzero_val: ncvar = what[11:]
    #
    ndim = len(dataset[ncvar].dims)
    if l_fill_val:
        # Mask is constructed out of variable and its missing value
        if not ndim in [3,4]: MsgExit(ncvar+' is expected to have 3 or 4 dimensions')
        if ndim==3: xmsk = 1 - nmp.isnan(dataset[ncvar][0,:,:])
        if ndim==4: xmsk = 1 - nmp.isnan(dataset[ncvar][0,0,:,:])
    elif l_nonzero_val:
        # Mask is constructed out of variable and where it is not 0
        if not ndim in [3,4]: MsgExit(ncvar+' is expected to have 3 or 4 dimensions')
        if ndim==3: xmsk = dataset[ncvar][0,:,:] > 0
        if ndim==4: xmsk = dataset[ncvar][0,0,:,:] > 0
    else:
        # Mask is read in mask file...
        if   ndim==2: xmsk = dataset[ncvar][:,:]
        elif ndim==3: xmsk = dataset[ncvar][0,:,:]
        elif ndim==4: xmsk = dataset[ncvar][0,0,:,:]
        else: MsgExit('FIX ME! Mask dataset has a weird number of dimensions:'+str(ndims))


 *** what we use to define model land-sea mask:
    => "salt" in dataset 



In [51]:
xmsk.values

array([[36.61352 , 36.612816, 36.61179 , ..., 36.320915, 36.320923,
        36.320793],
       [36.613426, 36.61243 , 36.61172 , ..., 36.32187 , 36.321644,
        36.32133 ],
       [36.613014, 36.612217, 36.612053, ..., 36.32278 , 36.322376,
        36.32179 ],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)

In [52]:
    ncvar = what
    if l_fill_val: ncvar = what[11:]
    if l_nonzero_val: ncvar = what[11:]

In [62]:
truc=dataset[ncvar][0,:,:].values > 0

In [63]:
truc

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [56]:
data=dataset[ncvar][0,:,:]

In [57]:
data.shape

(1489, 1491)

In [64]:
mask=truc.astype(int)

In [65]:
mask

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [68]:
import matplotlib.pyplot as plt 
plt.pcolormesh(mask)
plt.savefig('plot_mask.png')

NameError: name 'fig' is not defined